In [25]:
import sys
import os

os.getcwd()

'c:\\Users\\gabri\\Documents\\CMOR492-DWS\\optimization_models'

In [26]:
sys.path.append("C:\\Users\\gabri\\Documents\\CMOR492-DWS")

In [2]:
sys.path.append("D:\\Users\\gabri\\Documents\\Distributed Water System Modeling Spring 2025\\CMOR492-DWS")

In [2]:
sys.path.append("/Users/danielsuarez/Documents/Documents - Daniel’s MacBook Pro/Academic/Spring2025/CMOR492-DWS/")

In [27]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import numpy as np
from network_construction.network import source_treatment, get_Utown
import json
import ast

In [28]:
G = get_Utown()
source_nodes, treatment_nodes = source_treatment(G)

In [ ]:
### MODEL PARAMETERS

Path = {}  # Set of shortest paths from each source node i to each treatment node j
NLinks = {}  # Number of edges in each path
L = {}  # Length of each path (distance)

for i in source_nodes:
    for j in treatment_nodes:
        path = nx.shortest_path(G, source=i, target=j, weight='length')
        Path[i, j] = path
        NLinks[i, j] = len(path)-1
        L[i, j] = nx.path_weight(G, path, weight='length')
LE = {e: G.edges[e]['length'] for e in G.edges}  # Length of edge e
EL = {v: G.nodes[v]['elevation'] for v in G.nodes}  # Elevation of node v


D = [0.2, 0.25, 0.3, 0.35, 0.40, 0.45]  # Pipe diameters
CP = {0.05: 8.7, 0.06: 9.5, 0.08: 11,
                       0.1: 12.6, 0.15: 43.5, 0.2: 141,
                       0.25: 151, 0.3: 161, 0.35: 180,
                       0.4: 190, 0.45: 200}  # Cost per unit of pipe


SR_base = {}  # Production at Source node i
CAP = {}  # Capacity at treatment node j

for node in source_nodes:
    G.nodes[node]['production'] = .17 * 2
    SR_base[node] = .17 * 2

total_flow = sum(SR_base.values()) ########## ??? 

for node in treatment_nodes:
    G.nodes[node]['capacity'] = 100
    CAP[node] = 100

Vmin = 0.6 * 60
Vmax = 3 * 60

CE = 25  # Cost of Excavation
CB = 6  # Cost of Bedding
TR = 44000  # Fixed Cost of Treatment Plant
TRFlow = 100  # Variable Cost of Treatment
PICost = 30

PF = {'0.05': 8.7, '0.06': 9.5, '0.08': 11,
                       '0.1': 12.6, '0.15': 43.5, '0.2': 141,
                       '0.25': 151, '0.3': 161, '0.35': 180,
                       '0.4': 190, '0.45': 200}  # Fixed Cost of Piping

CT = 10000  # Cost of trucking
M = 1e6

Smin = 0.01
Smax = 0.1
W = 0.5  # Buffer Width

# time parameters
annual_rate = 0.05
years = 5
#compounded_rate = (1 + annual_rate) ** years
discount_factor = 1 / ((1 + annual_rate) ** years)

In [30]:
# helper
# Edge paths setup for edge activation constraint
ePath = {}  # Use this for Edge Activation Constraint
for e, p_ in Path.items():
    ePath[e] = [(p_[l - 1], p_[l]) for l in range(1, len(p_))]

# Helper function for flow definition
def is_sublist(short_list, long_list):
    for i in range(len(long_list) - len(short_list) + 1):
        if long_list[i:i + len(short_list)] == short_list:
            return True
    return False

In [31]:
### CONTEXT PARAMETERS

### T=0 PARAMETERS

with (open("context/x_sol.json", "r") as f):
    x_0 = {ast.literal_eval(k): v for k, v in json.load(f).items()}

with open("context/y_sol.json", "r") as f:
    y_0 = {ast.literal_eval(k): v for k, v in json.load(f).items()}

with open("context/z_sol.json", "r") as f:
    z_0 = {ast.literal_eval(k): v for k, v in json.load(f).items()}

with open("context/d_sol.json", "r") as f:
    d_0 = {ast.literal_eval(k): v for k, v in json.load(f).items()}

with open("context/el_sol.json", "r") as f:
    el_0 = {ast.literal_eval(k): v for k, v in json.load(f).items()}

In [32]:
# STOCHASTIC PARAMETERS
# Define scenarios and their probabilities
scenarios = ["low", "medium", "high"]
prob = {"low": 0.1, "medium": 0.5, "high": 0.4}  # Probability of each scenario

# Define scenario-dependent parameters (flow rates)
SR = {}
for node in source_nodes:
    SR[(node, 'present')] = SR_base[node]
    SR[(node, 'low')] = SR_base[node] * 0.8
    SR[(node, 'medium')] = SR_base[node] * 1.0
    SR[(node, 'high')] = SR_base[node] * 1.2

In [33]:
T = [0, 1, 2] # context, present, future
cases = [(0, 'context'), (1, 'present'), (2, 'low'), (2, 'medium'), (2, 'high')]


m = gp.Model()

x = m.addVars(Path.keys(), cases, vtype=GRB.BINARY, name='x')  # Path ij used
y = m.addVars(treatment_nodes, cases, vtype=GRB.BINARY, name='y')  # treatment at node j
z = m.addVars(G.edges, cases, vtype=GRB.BINARY, name='z')  # edge e used

d = m.addVars(G.edges, D, cases, vtype=GRB.BINARY, name='d')  # Pipe size s at edge e

a = m.addVars(G.edges, D, cases, vtype=GRB.BINARY, name='a')

r = m.addVars(source_nodes, cases,  vtype=GRB.CONTINUOUS, lb=0.0, name='r')  # flow handled at trucking at edge e

Q = m.addVars(G.edges, cases, vtype=GRB.CONTINUOUS, lb=0.0, name='Q')  # Flow in Edge e

el = m.addVars(G.nodes, cases, vtype=GRB.CONTINUOUS, name='el')  # Elevation at node el_v

p = m.addVars(Path.keys(), cases, vtype=GRB.CONTINUOUS, lb = 0.0, name='p')

c = m.addVars(G.nodes, cases,  vtype=GRB.BINARY, name='c')

# to linearize manning
alpha = m.addVars(G.edges, D, cases,  lb=0, name='alpha')
beta = m.addVars(G.edges, D, cases,  lb=0, name='beta')


m.update()


In [34]:
# Contextual Constraints 
x_0_c = m.addConstrs((x[i, j, 0, 'context'] == x_0[i, j] for i, j in Path.keys()), name='x_0')
y_0_c = m.addConstrs((y[j, 0, 'context'] == y_0[j] for j in treatment_nodes), name='y_0')
z_0_c = m.addConstrs((z[*e, 0, 'context'] == z_0[e] for e in G.edges), name='z_0')
d_0_c = m.addConstrs((d[*e, s, 0, 'context'] == d_0[*e, s] for e in G.edges for s in D), name='d_0')
el_0_c = m.addConstrs((el[u, 0, 'context'] == el_0[u] for u in G.nodes), name='el_0')

In [35]:
## Dynamic Constraints
# Infrascture Continuity
# Present stage (t=1) must respect context (t=0)
m.addConstrs((y[j, 1, "present"] >= y[j, 0, "context"] for j in treatment_nodes), name='treat_cont_0_1')
#m.addConstrs((x[i, j, 1, "present"] >= x[i, j, 0, "context"] for i, j in Path.keys()), name='path_cont_0_1')
#m.addConstrs((z[*e, 1, "present"] >= z[*e, 0, "context"] for e in G.edges), name='edge_cont_0_1')

# Future stage (t=2) must respect present stage (t=1)
for s in scenarios:
    m.addConstrs((y[j, 2, s] >= y[j, 1, "present"] for j in treatment_nodes), name=f'treat_cont_1_2_{s}')
#    m.addConstrs((x[i, j, 2, s] >= x[i, j, 1, "present"] for i, j in Path.keys()), name=f'path_cont_1_2_{s}')
#    m.addConstrs((z[*e, 2, s] >= z[*e, 1, "present"] for e in G.edges), name=f'edge_cont_1_2_{s}')


# a-constaint
m.addConstrs((a[*e, s, 1, 'present'] <= a [*e, s, 0, 'context'] + d[*e, s, 1, 'present'] for e in G.edges for s in D), name='a_constraint_present')
for scenario in scenarios:
    m.addConstrs((a[*e, s, 2, scenario] <= a[*e, s, 1, 'present'] + d[*e, s, 2, scenario] for e in G.edges for s in D), name=f'a_constraint_{scenario}')

#elevation change
for case in cases:
    m.addConstrs((gp.quicksum(d[*e, s, case[0], case[1]] for s in D) >= 
                  (0.5 * (c[e[0],case[0], case[1] ] + c[e[1], case[0], case[1]])) + (z[*e, case[0], case[1]] - 1) for e in G.edges), name=f'node_elevation1_{case}')

m.addConstrs((c[u, 1, 'present'] >= (el[u, 1, 'present'] - el[u, 0, 'context']) / M for u in G.nodes), name='cu_cons1_present')
m.addConstrs((c[u, 1, 'present'] >= (el[u, 0, 'context'] - el[u, 1, 'present']) / M for u in G.nodes), name='cu_cons2_present')

for scenario in scenarios:
    m.addConstrs((c[u, 2, scenario] >= (el[u, 2, scenario] - el[u, 1, 'present']) / M for u in G.nodes), name=f'cu_cons1_{scenario}')
    m.addConstrs((c[u, 2, scenario] >= (el[u, 1, 'present'] - el[u, 2, scenario]) / M for u in G.nodes), name=f'cu_cons2_{scenario}')

In [36]:
## Static Constraints

for case in cases:
    if case[0] == 0:  # context
        pass

    else:
        #NODE PRODUCTION MINUS RECOURSE
        m.addConstrs(
        (p[i, j, *case] >= (SR[(i, case[1])] * x[i, j, *case]) - r[i, *case] 
         for i, j in Path.keys()),
        name=f'node_prod_rec_{case[1]}'
        )
        m.addConstrs(
        (p[i, j, *case] >= 0
         for i, j in Path.keys()),
        name=f'node_prod_rec_{case[1]}'
        )


        #TREATMENT CAPACITY
        m.addConstrs(
        (gp.quicksum(p[i, j, *case] for i in source_nodes) <= CAP[j] * y[j, *case] 
         for j in treatment_nodes),
        name=f'treat_cap_{case[1]}'
    )

        #NODE ASSIGNMENT
        m.addConstrs(
        (gp.quicksum(x[i, j, *case] for j in treatment_nodes) == 1 
         for i in source_nodes),
        name=f'node_assign_{case[1]}'
        )

        #EDGE ACTIVATION
        m.addConstrs(
        (gp.quicksum(z[*e, *case] for e in ePath[i, j]) >= NLinks[i, j] * x[i, j, *case] 
         for i, j in Path),
        name=f'edge_activate_{case[1]}'
    )
        
        # FLOW DEFINITION
        m.addConstrs(
            (Q[*e, *case] == gp.quicksum(p[i, j, *case] for i, j in Path.keys() 
                                    if is_sublist(list((e[0], e[1])), Path[i, j])) 
            for e in G.edges),
            name=f'flow_def_{case[1]}'
        )



        # PIPE SIZING
        m.addConstrs(
            (gp.quicksum(a[*e, s, *case] for s in D) == z[*e, *case] for e in G.edges), name=f'pipe_size_a_{case[1]}')
        m.addConstrs(
            (gp.quicksum(d[*e, s, *case] for s in D) <= 1 for e in G.edges), name=f'pipe_sizing_d_{case[1]}')
        
        # Min/Max Slope 
        # MIN/MAX SLOPE
        m.addConstrs(
            (el[e[0], *case] - el[e[1], *case] >= 
            (LE[e] * Smin) - (M * (1 - z[*e, *case])) for e in G.edges),
            name=f'min_slope_{case[1]}'
        )
        m.addConstrs(
            (el[e[0], *case] - el[e[1], *case] <= 
            (LE[e] * Smax) + (M * (1 - z[*e, *case])) for e in G.edges),
            name=f'max_slope_{case[1]}'
        )

        # Flow Velocity Limit
        m.addConstrs(
        (Q[*e, *case] <= Vmax * gp.quicksum((np.pi / 8) * (s**2) * (a[*e, s, *case]) for s in D) 
         for e in G.edges),
        name=f'flow_vel_{case[1]}'
        )
        m.addConstrs(
        (Q[*e, *case] >= Vmin * gp.quicksum((np.pi / 8) * (s**2) * (a[*e, s, *case]) for s in D) 
         for e in G.edges),
        name=f'flow_vel_{case[1]}'
        )

        # below ground pipe 
        m.addConstrs(
        (el[u, *case] <= EL[u] for u in G.nodes),
        name=f'underground_{case[1]}'
        )
        

m.update()



In [37]:
# Objective Function

# First stage t = 1 
context_case = (0, 'context')
present_case = (1, 'present')
second_stage_cases = [(2, 'low'), (2, 'medium'), (2, 'high')]

# OBJECTIVE EXPR 1: TREATMENT COSTS
# First stage (present case)
treat_cost_present = gp.LinExpr()
for j in treatment_nodes:
    treat_cost_present.addTerms(TR, y[j, *present_case])
    treat_cost_present.addTerms(-1*TR, y[j, *context_case])
    for i in source_nodes:
        treat_cost_present.addTerms(TRFlow * SR[(i, 'present')], x[i, j, *present_case])

# Second stage (scenario cases)
treat_cost_scenarios = {}
for t, s in second_stage_cases:
    treat_cost_scenarios[s] = gp.LinExpr()
    for j in treatment_nodes:
        treat_cost_scenarios[s].addTerms(TR, y[j, t, s])
        treat_cost_scenarios[s].addTerms(-1*TR, y[j, *present_case])
        for i in source_nodes:
            treat_cost_scenarios[s].addTerms(TRFlow * SR[(i, s)], x[i, j, t, s])

# OBJECTIVE EXPR 2: EXCAVATION COSTS
# Function that works for any case
excav_cost_f = lambda u, v, case: gp.QuadExpr(CE * (((EL[u] - el[u, *case]) + (EL[v] - el[v, *case])) / 2) * 
                                             LE[u, v] * gp.quicksum(s + ((2*W) * d[u, v, s, *case]) for s in D))

# OBJECTIVE EXPR 3: BEDDING COSTS
# Function that works for any case
bed_cost_f = lambda u, v, case: gp.LinExpr(CB * LE[u, v] * 
                                          gp.quicksum(s + ((2*W) * d[u, v, s, *case]) for s in D))

# OBJECTIVE EXPR 4: PIPE COSTS
# Function that works for any case
pipe_cost_f = lambda u, v, case: gp.LinExpr(LE[u, v] * 
                                          gp.quicksum(CP[s] * d[u, v, s, *case] for s in D))

# First stage (present case) infrastructure costs
excav_bed_cost_present = gp.quicksum(excav_cost_f(u, v, present_case) + 
                                   bed_cost_f(u, v, present_case) + 
                                   pipe_cost_f(u, v, present_case) 
                                   for u, v in G.edges)

# Second stage (scenario cases) infrastructure costs
excav_bed_cost_scenarios = {}
for t, s in second_stage_cases:
    excav_bed_cost_scenarios[s] = gp.quicksum(excav_cost_f(u, v, (t, s)) + 
                                            bed_cost_f(u, v, (t, s)) + 
                                            pipe_cost_f(u, v, (t, s)) 
                                            for u, v in G.edges)

# OBJECTIVE EXPR 5: RECOURSE TRUCKING
# First stage (present case)
rec_cost_present = gp.LinExpr()
for i in source_nodes:
    rec_cost_present.addTerms(CT, r[i, *present_case])

# Second stage (scenario cases)
rec_cost_scenarios = {}
for t, s in second_stage_cases:
    rec_cost_scenarios[s] = gp.LinExpr()
    for i in source_nodes:
        rec_cost_scenarios[s].addTerms(CT, r[i, t, s])

# TOTAL OBJECTIVE: First stage + Expected second stage costs
# First stage total
first_stage_total = treat_cost_present + excav_bed_cost_present + rec_cost_present

# Second stage expected total
second_stage_total = gp.LinExpr()
for t, s in second_stage_cases:
    scenario_total = treat_cost_scenarios[s] + excav_bed_cost_scenarios[s] + rec_cost_scenarios[s]
    second_stage_total += prob[s] * scenario_total

# Set the objective
m.setObjective(first_stage_total + discount_factor * second_stage_total, GRB.MINIMIZE)




In [38]:
#m.setObjective(0, GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 9 5900HX with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 128810 rows, 140075 columns and 1075285 nonzeros
Model fingerprint: 0x5dc5d020
Model has 24216 quadratic objective terms
Variable types: 70300 continuous, 69775 integer (69775 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+06]
  Objective range  [2e+00, 5e+06]
  QObjective range [2e+01, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 48795 rows and 54607 columns (presolve time = 5s)...
Presolve removed 29921 rows and 56147 columns
Presolve time: 8.10s
Presolved: 119913 rows, 104952 columns, 958256 nonzeros
Variable types: 52696 continuous, 52256 integer (52256 binary)
Found heuristic solution: objective 9588501.8458
Deterministic concurrent LP optimizer: primal sim

In [39]:
def write_gurobidict_to_file(gurobidict, var_name, scenario, path_prefix="results\\"):
    """ 
    Dumps the keys and values in a Gurobi variable tupledict into a json file.

    Parameters
    ----------
    gurobidict : Gurobi tupledict
    The Gurobi tupledict we want to record in the file.

    var_name : str
    The name of the variable (e.g. "y"), to be used in the filename.

    period : Any
    The period for which the tupledict was optimized, to be used in the filename.

    path_prefix : str
    The path to the folder where you want to create the json files.
    """
    with open(path_prefix + var_name + "_sol_" + str(scenario) + ".json", "w") as f:
        json.dump({str(key) : gurobidict[key].X for key in gurobidict}, f)

In [ ]:
# Dump variables into json files
solutions_path = "solutions\\two_stage_stoch\\CT10000\\"
write_gurobidict_to_file(x, "x", "all", path_prefix=solutions_path)
write_gurobidict_to_file(y, "y", "all", path_prefix=solutions_path)
write_gurobidict_to_file(z, "z", "all", path_prefix=solutions_path)
write_gurobidict_to_file(a, "a", "all", path_prefix=solutions_path)
write_gurobidict_to_file(el, "el", "all", path_prefix=solutions_path)
write_gurobidict_to_file(r, "r", "all", path_prefix=solutions_path)
write_gurobidict_to_file(Q, "Q", "all", path_prefix=solutions_path)
write_gurobidict_to_file(p, "p", "all", path_prefix=solutions_path)
write_gurobidict_to_file(d, "d", "all", path_prefix=solutions_path)
write_gurobidict_to_file(c, "c", "all", path_prefix=solutions_path)